In [8]:
import pandas as pd
import numpy as np
import csv

## open file words save in a[] word,c dict{},b[] strength
f=open('C:/Users/hp/Desktop/explo/wordsc.csv', newline='')
reader = csv.reader(f)
a=[]
b=[]
c={}
for row in reader:
    a.append(row[0])
    b.append(int(row[2]))
    c[row[0]]=int(row[2])   
f.close()

## open file text save in d[] all sentences 
p=open('C:/Users/hp/Desktop/explo/text.csv', newline='')
reader=csv.reader(p)
d=[]
for row in reader:
    if(row[1]!=''):
        d.append(row[1])
p.close()


In [9]:
from textblob import TextBlob
### identifying postive as 1, neagative as -1 ,similar as 0

pl=[]
for i in range(len(d)):
    text = d[i]
    blob = TextBlob(text)
    if(blob.sentiment.polarity>0):
        pl.append(int(1))
    elif(blob.sentiment.polarity<0):
        pl.append(int(-1))
    else:
        pl.append(int(0))

In [10]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

## creating preference[] matrix , posw list posw[] and negw[] list.

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
preference=[]
posw=[]
negw=[]
pp=0
nn=0
flag=0
no_of_alter=4
experts=4

for i in range(len(d)):
    if(flag==no_of_alter**2-no_of_alter):
        flag=0
        posw.append(pp)
        negw.append(nn)
        pp=0
        nn=0
    s=d[i]
    p=[]
    k=[]
    p=tokenizer.tokenize(s)
    filtered_sentence = [] 
    for w in p: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
            
    for j in range(len(filtered_sentence)):
        for x in range(len(a)):
            if(filtered_sentence[j]==a[x]):
                if(pl[i]>0 or pl[i]==0):
                        k.append(c[a[x]])
                else:
                    k.append(int(-1*c[a[x]]))
                break
    if(k):
        if(k[0]>0 or k[0]==0):
            preference.append(max(k))
            pp+=len(k)
        else:
            preference.append(min(k))
            nn+=len(k)
    
    flag+=1

if(flag==no_of_alter**2-no_of_alter):
        flag=0
        posw.append(pp)
        negw.append(nn)
        pp=0
        nn=0

p=[]
k=[]

for i in range(len(preference)):
    preference[i]=preference[i]+5
    
print(posw,negw)
print(preference)


[12, 12, 12, 12] [2, 0, 0, 1]
[7, 3, 9, 10, 4, 9, 10, 9, 10, 6, 7, 6, 7, 6, 9, 10, 6, 8, 10, 10, 10, 6, 6, 7, 6, 6, 10, 9, 6, 9, 9, 9, 9, 7, 7, 6, 7, 7, 10, 9, 7, 9, 10, 10, 10, 7, 7, 3]


In [11]:
import numpy as np
import pandas as pd

## collective perefernce C[][]
weight=1/experts
a=(no_of_alter,no_of_alter)
c=[]
k=0
m=0
for i in range(no_of_alter*no_of_alter-no_of_alter):
    for j in range(m,len(preference),no_of_alter*no_of_alter-no_of_alter):
        k+=preference[j]
    c.append(k)
    m+=1
    k=0
    
C=np.zeros(a)
k=[]
m=0

for i in range(no_of_alter):
    for j in range(no_of_alter):
        if(i==j):
            k.append(0)
        else:
            k.append(c[m])
            m+=1
    C[i]=k
    k=[]
C=C*weight
print(C)

### table5

QGDD=[]
for i in range(no_of_alter):
    s=0
    for j in range(no_of_alter):
        s+=C[i][j]
    QGDD.append(s/no_of_alter)
print(QGDD)

RV={}
p_mx=np.max(C)        ##max no. in C
p_mn=np.min(c)*weight ## min no. in C
for i in range(no_of_alter):
    t=(p_mx-QGDD[i])/(p_mx-p_mn)
    RV[i+1]=t
print(RV)

ranking = sorted(RV.items(), key=lambda x: x[1])    
for i in range(no_of_alter):
    print(ranking[i][0])



[[0.   6.75 5.5  9.5 ]
 [9.5  0.   5.75 8.75]
 [9.75 9.5  0.   9.75]
 [6.5  6.75 5.5  0.  ]]
[5.4375, 6.0, 7.25, 4.6875]
{1: 1.0147058823529411, 2: 0.8823529411764706, 3: 0.5882352941176471, 4: 1.1911764705882353}
3
2
1
4


In [12]:
##table 6 way_1 for calclulating consensus
### table6 i,j are experts and p1,p2 .... be prefernce matrices

CN=[]
c=0
d=no_of_alter*no_of_alter-no_of_alter
for i in range(experts):
    for j in range(experts):
        if(j>i):
            for k in range(d):
                res=preference[i*d+k]-preference[j*d+k]
                res=res**2
                c+=res
            CN.append(int(np.sqrt(c)))
        res=0
        c=0
print(CN)

s=0
for i in range(len(CN)):
    s+=CN[i]
GCN=s/len(CN)
print(GCN)

[4, 4, 6, 3, 4, 3]
4.0


In [13]:
## table 7 way_2 for claculate consensensus

a=(experts,experts)
CS=np.zeros(a)
k=[]
for i in range(experts):
    for j in range(experts):
        if(i==j):
            k.append(0)
        else:
            a=posw[i]+posw[j]
            b=negw[i]+negw[j]
            p=((a-b)/(a+b))+1 
            c=p/2
            k.append(c)
    CS[i]=k
    k=[]
print(CS)

### calculate global consensus
GCS=(np.sum(CS))/(no_of_alter*no_of_alter-no_of_alter)
print(GCS)

[[0.         0.92307692 0.92307692 0.88888889]
 [0.92307692 0.         1.         0.96      ]
 [0.92307692 1.         0.         0.96      ]
 [0.88888889 0.96       0.96       0.        ]]
0.9425071225071225


In [14]:
print(GCN,GCS)
print("consensus reached in both consensus measuring approaches are quite high. Then,it is possible to accept the generated alternatives ranking as the final one.")
for i in range(no_of_alter):
    print(ranking[i][0])
print("number of computation",((experts*(no_of_alter**2-no_of_alter)*(len(preference)/experts))+(no_of_alter**2-no_of_alter)*experts))

4.0 0.9425071225071225
consensus reached in both consensus measuring approaches are quite high. Then,it is possible to accept the generated alternatives ranking as the final one.
3
2
1
4
number of computation 624.0
